# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/city_weather_data.csv")
df = df[["City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,57.31,86,75,17.27,US,1663870445
1,Norman Wells,65.2820,-126.8329,44.64,87,75,2.30,CA,1663870568
2,Vaini,-21.2000,-175.2000,75.56,87,91,17.11,TO,1663870590
3,Kapaa,22.0752,-159.3190,75.34,91,75,5.75,US,1663870615
4,Dengzhou,37.8086,120.7508,64.62,45,39,24.09,CN,1663870794
...,...,...,...,...,...,...,...,...,...
570,Namatanai,-3.6667,152.4333,77.16,79,52,3.58,PG,1663871082
571,Kamenka,51.3223,42.7678,57.52,96,100,3.40,RU,1663871082
572,Neijiang,29.5835,105.0622,65.70,93,100,7.94,CN,1663871083
573,Stornoway,58.2093,-6.3865,53.04,86,40,8.05,GB,1663871083


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
loc = df[["Lat","Lng"]]
hum = df["Humidity"]

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
hum_heatmap = gmaps.figure()
hum_heatmap.add_layer(gmaps.heatmap_layer(loc, weights=hum))
hum_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather_df = df.loc[(df["Max Temp"] >= 75) & (df["Max Temp"] <= 85) & (df["Wind Speed"] < 10) & (df["Cloudiness"] < 10) & (df["Humidity"] < 50)]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,Gazli,40.1333,63.4500,79.66,11,0,6.38,UZ,1663870834
248,Saint George,37.1041,-113.5841,83.70,39,0,4.61,US,1663870575
283,Abu Kamal,34.4506,40.9171,83.35,27,0,5.75,SY,1663870930
329,Rāvar,31.2656,56.8055,79.99,14,0,4.07,IR,1663870953
352,Chihuahua,28.5000,-106.0000,77.83,44,0,8.93,MX,1663870963
390,Los Lunas,34.8062,-106.7334,81.48,47,0,1.01,US,1663870987
398,Bam,29.1060,58.3570,84.33,12,1,6.71,IR,1663870991
539,Illapel,-31.6308,-71.1653,79.05,13,0,6.71,CL,1663871066


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key}

lat = ideal_weather_df["Lat"].tolist()
lng = ideal_weather_df["Lng"].tolist()
hotel_name = []

for i in range(len(lat)):
    params["location"] = f"{lat[i]}, {lng[i]}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    data = response.json()
    try:
        hotel_name.append(data["results"][0]["name"])
    except:
        hotel_name.append("No hotels")

In [8]:
ideal_weather_df["Hotel Name"] = hotel_name
ideal_weather_df

C:\Users\akgso\AppData\Local\Temp\ipykernel_2380\4222120791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel Name"] = hotel_name


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
87,Gazli,40.1333,63.4500,79.66,11,0,6.38,UZ,1663870834,No hotels
248,Saint George,37.1041,-113.5841,83.70,39,0,4.61,US,1663870575,"The Advenire, Autograph Collection"
283,Abu Kamal,34.4506,40.9171,83.35,27,0,5.75,SY,1663870930,البوكمال
329,Rāvar,31.2656,56.8055,79.99,14,0,4.07,IR,1663870953,Hotel sperlous
352,Chihuahua,28.5000,-106.0000,77.83,44,0,8.93,MX,1663870963,No hotels
390,Los Lunas,34.8062,-106.7334,81.48,47,0,1.01,US,1663870987,Quality Inn
398,Bam,29.1060,58.3570,84.33,12,1,6.71,IR,1663870991,Azadi parsian hotel
539,Illapel,-31.6308,-71.1653,79.05,13,0,6.71,CL,1663871066,Hotel Ortiz de Rozas


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
hum_heatmap.add_layer(markers)
# Display figure
hum_heatmap

Figure(layout=FigureLayout(height='420px'))